# MNIST

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [4]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Dense, Reshape, BatchNormalization, Activation, Conv2DTranspose
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
def build_generator(inputs, image_size):
    